#------------STARTING POINT---------------

In [13]:
# !pip install --upgrade sagemaker

In [6]:
!pip show  sagemaker

Name: sagemaker
Version: 2.39.1
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: None
License: Apache License 2.0
Location: /usr/local/lib/python3.7/site-packages
Requires: google-pasta, importlib-metadata, packaging, numpy, boto3, attrs, smdebug-rulesconfig, pandas, protobuf3-to-dict, pathos, protobuf
Required-by: 


In [14]:
%%time
import boto3
import re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
import os 
import urllib.request
import boto3

#Download and Unzip
# !wget https://yourDatasource
# !unzip https://yourdatasource.zip


role = get_execution_role()
# bucket = sagemaker.Session().default_bucket()
bucket = "prj-ml"

training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

def upload_to_s3(channel, Path):
    s3_path_to_data = sagemaker.Session().upload_data(bucket=bucket, 
                                                  path=Path, 
                                                  key_prefix=channel)

# data copy to s3
s3_train_key = "chest-xray-img-classfcn-full-training/train"
s3_validation_key = "chest-xray-img-classfcn-full-training/validation"
s3_train = 's3://{}/{}/'.format(bucket, s3_train_key)
s3_validation = 's3://{}/{}/'.format(bucket, s3_validation_key)


upload_to_s3(s3_train_key, './chest_xray/train/')

upload_to_s3(s3_validation_key, './chest_xray/val/')


The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


CPU times: user 36 s, sys: 5.86 s, total: 41.9 s
Wall time: 7min 12s


In [7]:
import os
import sagemaker
from sagemaker.tensorflow import TensorFlow

In [15]:
ON_SAGEMAKER_NOTEBOOK = True

sagemaker_session = sagemaker.Session()
if ON_SAGEMAKER_NOTEBOOK:
    role = sagemaker.get_execution_role()
else:
    role = "[YOUR ROLE]"


bucket = "prj-ml"
key = "chest-xray-img-classfcn-full-training"
key_output = "output"                   # Path from the bucket's root to the dataset
train_instance_type='ml.m5.large'      # The type of EC2 instance which will be used for training
deploy_instance_type='ml.m5.large'     # The type of EC2 instance which will be used for deployment
hyperparameters={
    "learning_rate": 1e-4,
    "decay": 1e-6
}

train_input_path = "s3://{}/{}/train/".format(bucket, key)
validation_input_path = "s3://{}/{}/validation/".format(bucket, key)

In [16]:
train_use_spot_instances=True
train_max_run=3600
train_max_wait = 3600 if train_use_spot_instances else None

In [ ]:
estimator = TensorFlow(
  entry_point="model.py",             # Your entry script
  role=role,
  framework_version="1.14.0",               # TensorFlow's version
  hyperparameters=hyperparameters,
  training_steps=100,
  evaluation_steps=100,
  instance_count=1,                   # "The number of GPUs instances to use"
  instance_type=train_instance_type,
  use_spot_instances=train_use_spot_instances,  
  max_run=train_max_run,
  max_wait=train_max_wait,
  py_version="py3",  # now required    
)

print("Training ...")
estimator.fit({'training': train_input_path, 'validation': validation_input_path})

print("Deploying ...")
predictor = estimator.deploy(initial_instance_count=1, instance_type=deploy_instance_type)

print("Predictor endpoint name : %s" % predictor.endpoint)


Training ...
2021-05-09 21:21:06 Starting - Starting the training job...
2021-05-09 21:21:29 Starting - Launching requested ML instancesProfilerReport-1620595265: InProgress
......
2021-05-09 21:22:29 Starting - Preparing the instances for training......
2021-05-09 21:23:30 Downloading - Downloading input data......
2021-05-09 21:24:30 Training - Downloading the training image../usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-pa